In [ ]:
from langgraph.graph import StateGraph, START , END
from langchain_openai import ChatOpenAI
from typing import TypedDict
from dotenv import load_dotenv

In [36]:
load_dotenv()

True

In [37]:
import os
api_key = os.environ.get("PERPLEXITY_API_KEY")


In [38]:
model = ChatOpenAI(
    api_key=api_key,
    base_url="https://api.perplexity.ai",
    model="sonar"  # or another supported Perplexity model
)


In [39]:
#create a state  
class LLMState(TypedDict):

    question : str
    answer : str


In [40]:
def llm_qa(state : LLMState) -> LLMState: 

    # extract the question from state 
    question = state['question']

    # form a prompt 
    prompt = f'Answer the following question {question}'

    #ask the que to llm 
    answer = model.invoke(prompt).content

    #update the answer in the state 
    state['answer'] = answer 

    return state

In [41]:
#create our graph 
graph = StateGraph(LLMState)

# add nodes 
graph.add_node("llm_qa", llm_qa)

# add edges 
graph.add_edge(START ,'llm_qa' )
graph.add_edge('llm_qa' , END)

# compile our graph 
workflow = graph.compile()


In [42]:
#execute 
initial_state = {'question':'How far is moon from earth'}

final_state = workflow.invoke(initial_state)

print(final_state['answer'])

The Moon is on average about 384,400 kilometers (238,855 miles) away from Earth. However, because the Moon’s orbit around Earth is elliptical, this distance varies from about 356,470 kilometers (221,500 miles) at its closest (perigee) to about 405,600 kilometers (252,000 miles) at its farthest (apogee)[1][2][3][4].

This variation is due to the elliptical shape of the orbit, with an eccentricity of approximately 0.05, meaning the orbit is slightly oval rather than a perfect circle[3]. Light takes about 1.3 seconds to travel from the Moon to Earth because of this average distance[3].


In [43]:
model.invoke('how far is sun from earth').content

"The Sun is on average about 93 million miles (150 million kilometers) away from Earth. This distance is also called 1 Astronomical Unit (AU), a standard unit used in astronomy to describe distances within the solar system[1][2][4][5].\n\nBecause Earth's orbit around the Sun is slightly elliptical, the distance varies throughout the year. At its closest point (perihelion), the Sun is about 91.4 million miles (147.1 million kilometers) away, and at its farthest point (aphelion), it is about 94.5 million miles (152.1 million kilometers)[1]. Currently, as of August 2025, the distance is about 151.7 million kilometers (around 1.014 AU)[3].\n\nLight from the Sun takes roughly 8 minutes and 20-26 seconds to reach Earth, traveling at about 186,000 miles (300,000 kilometers) per second[1][3][5]."